# Track Tip Trajectories for Folder of Tip Logs
Tim Tyree<br>
12.17.2020

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

#automate the boring stuff
# from IPython import utils
import time, os, sys, re
import dask.bag as db
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
    
#load the libraries
from lib import *
from lib.utils.operari import *

%autocall 1
%load_ext autoreload
%autoreload 2
# awareness&chill

Automatic calling is: Smart


In [2]:
#make plots work in darkmode
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# compute trajectories for folder of tip logs

In [3]:
# fn_out = run_routine(file)

In [4]:
# #TODO: make save_fn replace Log with trajectories because apparently it isn't
# # save_fn = input_file_name.replace('/Log','/trajectories').replace('log.csv', f'traj_sr_{sr}_mem_{mem}.csv')
# input_file_name = file_name_list[0]
# sr=400
# mem=2
# save_fn = os.path.abspath(input_file_name).replace('/Log','/trajectories').replace('log.csv', f'traj_sr_{sr}_mem_{mem}.csv')

# print(save_fn)

In [35]:
# #find file interactively
# print("please select a file from within the desired folder.")
# file = search_for_file()

please select a file from within the desired folder.
File: /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.5_archive/Log/ic_200x200.004.11_log.csv


In [36]:
#9 samples LT = 0
# file =  '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/ds_5_param_set_8/birth-death-rates/33_ds_5_sigma_1.5_threshold_0.6__traj_sr_40_mem_2_bdrates.csv'
# file = "/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.4_archive/Log/ic_200x200.002.31_log.csv"
#>60 samples LT = 2
# file = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-1/ds_5_param_set_8/birth-death-rates/ic_200x200.117.21_ds_5_sigma_1.5_threshold_0.6_sr_50_mem_2_tmin_100_LT_2_bdrates.csv'
#change to the directory containing file
# file = "/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.6_archive/Log/ic_200x200.001.11.csv"
folder_name = os.path.dirname(file)


#find folder interactively. Crashes jupyter's root kernel :(
# from tkinter import Tk
# from tkinter.filedialog import askdirectory
# path = askdirectory(title='Select Folder') # shows dialog box and return the path
# print(path) 

# get all .csv files in the current working directory
os.chdir(folder_name)
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [f for f in file_name_list if is_csv(f)]
# remove all files with 'threshold'
# file_name_list = [f for f in file_name_list if f.find('threshold')==-1]

print(len(file_name_list))

171


In [37]:
def produce_one_csv(list_of_files, file_out):
   # Consolidate all csv files into one object
   result_obj = pd.concat([pd.read_csv(file) for file in list_of_files])
   # Convert the above object into a csv file and export
   result_obj.to_csv(file_out, index=False, encoding="utf-8")

In [38]:
def run_routine(ic):
    output_file_name = preprocess_log(ic)
    return output_file_name

In [39]:
b = db.from_sequence(file_name_list, npartitions=10).map(run_routine)
start = time.time()
retval = list(b)
print(f"run time for generating birth-death rates from file_name_list: {time.time()-start:.2f} seconds.")
beep(3)

Frame 12785: 2 trajectories present.
run time for generating birth-death rates from file_name_list: 9882.84 seconds.


0

In [11]:
# ic = file_name_list[0]

In [13]:
# input_file_name = "/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.4_archive/Log/ic_200x200.001.11_log.csv"
# output_file_name = preprocess_log(ic)

Frame 3811: 4 trajectories present.


# unwrap each trajectory in folder of trajectories

In [68]:
#find file interactively
print("please select a file from within the desired folder.")
file = search_for_file()

please select a file from within the desired folder.
File: /home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.6_archive/trajectories/ic_200x200.001.22_traj_sr_400_mem_2.csv


In [69]:
# load trajectories
from lib.routines.compute_msd import *

In [70]:
folder_name = os.path.dirname(file)
folder_name_output = folder_name.replace('trajectories','trajectories_unwrapped')
# get all .csv files in the current working directory
os.chdir(folder_name)
retval = !ls
file_name_list = list(retval)
# check each file if it ends in .csv before merging it
def is_csv(file_name):
    return file_name[-4:]=='.csv'
file_name_list = [os.path.join(folder_name,f) for f in file_name_list if is_csv(f)]
# remove all files with 'threshold'
# file_name_list = [f for f in file_name_list if f.find('threshold')==-1]

print(len(file_name_list))

171


In [71]:
def run_routine(input_file_name):
    output_file_name = input_file_name.replace('trajectories','trajectories_unwrapped')
    unwrap_trajectories(input_file_name, output_file_name)
    return output_file_name

In [72]:
b = db.from_sequence(file_name_list, npartitions=10).map(run_routine)
start = time.time()
retval = list(b);
print(f"run time for generating birth-death rates from file_name_list: {time.time()-start:.2f} seconds.")
beep(3)

run time for generating birth-death rates from file_name_list: 8639.31 seconds.


0

In [ ]:

#select the longest n trajectories
n_tips = 1#15
df = pd.read_csv(output_file_name)
s = df.groupby('particle').t.count()
s = s.sort_values(ascending=False)
pid_longest_lst = list(s.index.values[:n_tips])
# d = df[df.particle==pid_longest]
# print(pid_longest)
# print(s.head())
# pid_longest_lst = s.head(n_tips).values
df_traj = pd.concat([df[df.particle==pid] for pid in pid_longest_lst])
assert ( (np.array(sorted(set(df_traj['particle'].values)))==np.array(sorted(pid_longest_lst))).all())
#compute ensemble mean squared displacement
emsd = trackpy.motion.emsd(df_traj, mpp=1., fps=1.,max_lagtime=40000)
#save results
output_file_name_msd = output_file_name.replace("trajectories/","msd/").replace("traj_",f"emsd_ntips_{n_tips}")
emsd.to_csv(output_file_name_msd, index=False)

In [ ]:
saving = False
savefig_fn = os.path.basename(output_file_name_msd).replace('.csv','.svg')
savefig_folder = os.path.join(nb_dir,'Figures/msd')
fontsize =22
figsize=(6,4)
DS = 5/200 #cm per pixel
DT = 1. #ms per frame
x_values = emsd.index.values*DT/10**3 #lag in seconds
y_values = emsd.values*DS**2 #msd

fig, ax = plt.subplots(figsize=figsize)
ax.plot(x_values,y_values)
ax.set_xlabel('lag (s)', fontsize=fontsize)
ax.set_ylabel(r'MSD (cm$^2$)', fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)
ax.tick_params(axis='both', which='minor', labelsize=0)
if not saving: 
    plt.show() 
else:
    plt.tight_layout()
    os.chdir(savefig_folder)
    plt.savefig(savefig_fn, dpi=300)
    print(f"saved figure in \n\t{savefig_fn}")